In [ ]:
#| default_exp shallow

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
from fastcore.test import *
import numpy as np

# Shallow water corrections

> Addressing additional resistance caused by changes to viscosity and additional sinkage in shallow water

When ships pass through shallow water, they experience higher resistance due to changes in the viscous friction and the impact of squatting or sinkage in the water. These effects are due to the water passing under the ship being constricted and thus having a higher velocity. The change in velocity results in a difference in the buoyancy of the ship in accordance with the Bernoulli principle, which states that 

$$P +\frac{1}{2} \rho v^2 + \rho g h = \textrm{Constant}$$

Where $P$ is pressure, $\rho$ is water density, $v$ is the speed through water,  $g$ is gravity, and $h$ is the distance from the reference plane. As can be seen, an increase in water velocity must be balanced by a change in at least one of the other two variables. Changes in pressure result in the ship's buoyancy being reduced, causing it to sink slightly or squat. As the coefficient of friction experienced by the vessel is proportional to the square of the speed through water, the frictional resistance on the ship increases. A secondary effect is that the sinkage exposes an additional surface area of the ship to the water, increasing the friction effects further. Finally, as the ship's hull gets closer to the sea floor, the boundary layer between the hull and the free-flowing water thickens and becomes more turbulent increasing resistance. As a result of the additional resistance, the engine is required to deliver more power to maintain speed.

As such, when calculating the performance of a ship in shallow water, care needs to be taken to adjust for the effects this produces or risk misinterpreting the performance results.

## Importing the module

The module is called `shallow` import as

`from pyseatrials.shallow import *`

## Raven shallow water correction

Although there are several correction approaches to correcting for the effect of shallow water this library uses the method introduced by Raven in 2016 and described fully in [Raven 2019](https://doi.org/10.1016/j.oceaneng.2019.106343). The correction has two component resistance due to changes in viscous friction, resistance due to sinkage.

Consider the following

$R_{V, \,\textrm{deep}} = C_v'\frac{1}{2} \rho  V_S^2 S$

Is the viscous friction in deep water, where $C_v'$ is the coeficient of viscous friction, $\rho$ is the density of water, $V_S$ is the speed through water, and $S$ is the wetted surface area

The correction due to the changes in viscous resistance are given as 

$$ R_{V} = R_{V, \,\textrm{deep}} 0.57 (\frac{T_M}{h})^{1.79},$$

Where, $T_M$ is the draught at midship and $h$ is the water depth. 

The sinkage is calculated using the following method,

$$d(\textrm{sinkage}) = 1.46 \frac{\nabla}{L_\textrm{pp}^2}\left[ 
\frac{\textrm{Fr}_\textrm{h}^2}{1 - \textrm{Fr}_\textrm{h}^2} -
 \frac{\textrm{Fr}_\textrm{hd}^2}{1 - \textrm{Fr}_\textrm{hd}^2}
\right],$$

where L_\textrm{pp} is the length between perpendiculars, $\nabla$ is $\nabla = L_\textrm{pp} B T_M C_B$, and $B$ is the ships beam (m), $C_B$ is the dimensionless block coefficient, $\textrm{Fr}_\textrm{h}$ is the Froude number where $h = 0.3 L_\textrm{pp}$, $\textrm{Fr}_\textrm{hd}$ is the Froude number where $h$ is water depth. It should be noted that $d(\textrm{sinkage}) \geq 0 $. 

Additional displacement due to sinkage is then found such that

$$\delta \nabla =  \begin{cases}
d(\textrm{sinkage})\frac{A_W}{\nabla} & \text{if } d(\textrm{sinkage})\frac{A_W}{\nabla} < 0.05 \\
0.05 & \text{if } d(\textrm{sinkage})\frac{A_W}{\nabla} \geq 0.05
\end{cases} $$

Where $A_W$ is the water plane area. With $\delta nabla$ now known we find the sinkage value using

$$\textrm{rsink} = (1- \delta \nabla)^\frac{2}{3} $$

We then correct the power shallow water power to deep water power using 

$$P_\textrm{D,deep} = \frac{P_\textrm{D, shallow}}{\textrm{rsink}} - \frac{\Delta R_V V_S}{\eta_{Did}}$$

Finally we perform a check to ensure that

$$R_{V, \,\textrm{deep}}  \leq  \frac{P_\textrm{D,deep} \eta_\textrm{Did}}{V_s}$$

if the above condition is not satisfied $R_{V, \,\textrm{deep}}$ is set to the upper limit and the process is repeated.




In [ ]:
#| export
def shallow_water_correction(Cv_prime: float, rho: float, Vs: float, S: float, T_M: float, h: float, 
                             L_pp: float, B: float, C_B: float, P_D_shallow: float, eta_Did: float, R_V_deep=None) -> tuple[float, float, float]:
    """
    Perform Raven corrections for shallow water performance

    """
    # Calculate viscous friction in deep water if not provided
    if R_V_deep is None:
        R_V_deep = Cv_prime * 0.5 * rho * Vs**2 * S
    
    # Calculate the viscous resistance correction
    R_V = R_V_deep * 0.57 * (T_M / h)**1.79
    
    # Calculate the sinkage
    displacement = L_pp * B * T_M * C_B
    Fr_h = Vs / np.sqrt(9.81 * 0.3 * L_pp)
    Fr_hd = Vs / np.sqrt(9.81 * h)
    sinkage = 1.46 * displacement / L_pp**2 * (Fr_h**2 / (1 - Fr_h**2) - Fr_hd**2 / (1 - Fr_hd**2))
    sinkage = max(sinkage, 0)
    
    # Calculate additional displacement due to sinkage
    A_W = L_pp * B
    #delta displacement is written like this to allow vectorisation
    delta_displacement = np.minimum(sinkage * A_W / displacement, 0.05)
    
    # Calculate rsink
    rsink = (1 - delta_displacement)**(2/3)
    
    # Calculate deep water power
    P_D_deep = (P_D_shallow / rsink) - (R_V * Vs / eta_Did)
    
    # Check if the condition is met
    condition_met = R_V_deep <= P_D_deep * eta_Did / Vs
    
    # If the condition is not met, set R_V_deep to the upper limit and repeat the process
    if not condition_met:
        R_V_deep = P_D_deep * eta_Did / Vs
        P_D_deep, sinkage, R_V = shallow_water_correction(Cv_prime, rho, Vs, S, T_M, h, L_pp, B, C_B, P_D_shallow, eta_Did, R_V_deep)
    
    return P_D_deep, sinkage, R_V


The example below shows a simple use case

In [ ]:
# Define input parameters
Cv_prime = 0.9
rho = 1025
Vs = 10
S = 500
T_M = 8
h = 6
L_pp = 100
B = 15
C_B = 0.8
P_D_shallow = 20000
eta_Did = 0.7

# Calculate deep water power and check condition for deep water performance
P_D_deep, sinkage, R_V = shallow_water_correction(Cv_prime, rho, Vs, S, T_M, h, L_pp, B, C_B, P_D_shallow, eta_Did)

# Print results
print(f"Deep water power needed: {P_D_deep:.2f} W")
if R_V <= P_D_deep * eta_Did / Vs:
    print("Condition for deep water performance is met.")
else:
    print("Condition for deep water performance is not met.")

Deep water power needed: 299805857.12 W
Condition for deep water performance is met.


In [ ]:
#| hide
test_case1 = {
    'Cv_prime': 0.0025,
    'rho': 1025,
    'Vs': 10,
    'S': 500,
    'T_M': 5,
    'h': 50,
    'L_pp': 100,
    'B': 20,
    'C_B': 0.85,
    'P_D_shallow': 250000,
    'eta_Did': 0.7
}

P_D_deep1, sinkage1, R_V1 = shallow_water_correction(**test_case1)
test_close(P_D_deep1, 256383, eps=1)
test_close(sinkage1, 0.32, eps=1e-3)



# Test case 2: Another example with different parameters
##
##This result seems a bit strange
test_case2 = {
    'Cv_prime': 0.003,
    'rho': 1025,
    'Vs': 15,
    'S': 800,
    'T_M': 10,
    'h': 25,
    'L_pp': 150,
    'B': 30,
    'C_B': 0.9,
    'P_D_shallow': 500000,
    'eta_Did': 0.6
}

P_D_deep2, sinkage2, R_V2 = shallow_water_correction(**test_case2)
test_close(P_D_deep2, 529282, eps=1)
test_close(sinkage2, 0, eps=1e-6)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()